In [1]:
!pip install "ray[client,default]"

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 39.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 47.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [opentelemetry-exporter-prometheus]


In [2]:
import ray
ray.init()

2025-08-08 11:10:51,416	INFO worker.py:1606 -- Using address ray://ai-starter-kit-kuberay-head-svc:10001 set in the environment variable RAY_ADDRESS
2025-08-08 11:10:51,434	INFO client_builder.py:242 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.
2025-08-08 11:10:53,738	WARNING utils.py:1280 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.48.0
    Python: 3.12.9
This process on Ray Client was started with:
    Ray: 2.48.0
    Python: 3.12.10



Python version:,3.12.9
Ray version:,2.48.0
Dashboard:,http://10.244.0.9:8265


In [3]:
@ray.remote
def cluster_resources():
  return ray.cluster_resources()

print(ray.get(cluster_resources.remote()))

{'node:10.244.0.9': 1.0, 'CPU': 2.0, 'memory': 9000000000.0, 'object_store_memory': 1823143525.0, 'node:__internal_head__': 1.0, 'node:10.244.0.10': 1.0}


In [4]:
# here is the reference https://docs.ray.io/en/latest/serve/llm/serving-llms.html#deployment-through-llmrouter

from ray import serve
from ray.serve.llm import LLMConfig, build_openai_app

llm_config = LLMConfig(
    model_loading_config=dict(
        model_id="qwen-0.5b",
        model_source="Qwen/Qwen2.5-0.5B-Instruct",
    ),
    deployment_config=dict(
        autoscaling_config=dict(
            min_replicas=1, max_replicas=2,
        )
    ),
    # The accelerator_type and tensor_parallel_size are removed
    # or commented out as they are not applicable to the M1 chip
    engine_kwargs=dict(
        # tensor_parallel_size=2, # Removed for single-device inference
    ),
)

app = build_openai_app({"llm_configs": [llm_config]})
serve.run(app, blocking=True)

ModuleNotFoundError: No module named 'vllm'